In [ ]:
library(data.table)

In [ ]:
setwd(project_directory) # set wd to project directory containing all the sub folder

### Process HDST data

Correct coordinates and compress (long format) HDST data

In [7]:
prep_hdst_data=function(dir,sample,genes,plot=FALSE){
    adjust.y=function(bc){
      spl=as.numeric(unlist(strsplit(as.character(bc),"x")))
      x=spl[1]
      y=spl[2]
      if (y <= 255){
        y=y+529
      }else if (y >= 530){
        y=y-529
      }
      return(paste0(x,"x",y))
    }
    if (file.exists(paste0(dir,"/",sample,"_filtered_red_ut.tsv"))){
        dat_long_filt_annot_red=fread(paste0(dir,"/",sample,"_filtered_red_ut.tsv"))
        pl=ggplot(dat_long_filt_annot_red[sample(1:nrow(dat_long_filt_annot_red),40000)],aes(y=spot_px_y,x=spot_px_x))+geom_point(size=0.5)+coord_fixed()
        png(paste0(dir,"/",sample,"_filtered_corrected.png"),height=500,width=700)
        print(pl)
        dev.off()
        return(dat_long_filt_annot_red)
    }
    
    dat=fread(paste0(dir,"/",sample,"_filtered.tsv.gz"))
    sel_bc=fread(paste0(dir,"/",sub("_unmodgtf|_modgtf","",sample),"_barcodes_under_tissue.tsv"))
    if (any(grepl("ENS",dat$V1))){
        message("Running in transpose mode")
        chunks=unique(c(seq(from=2,to=ncol(dat),by=1000),ncol(dat)))
        steps=as.data.table(cbind(from=chunks[-length(chunks)],to=chunks[-1]-1))
        steps[nrow(steps),to:=to+1,]
    
        dat_long=data.table()
        for (i in 1:nrow(steps)){
            cat(paste0(i," "))
            from=steps[i]$from
            to=steps[i]$to
            dat_long_tmp=melt(dat[,c(1,from:to),with=FALSE],id.vars=c("V1"),variable.name = "bc_old")
            dat_long_tmp[,bc:=adjust.y(bc_old[1]),by=bc_old]
            dat_long=rbindlist(list(dat_long,dat_long_tmp[value>0]))
            }
        setnames(dat_long,c("V1","value"),c("ensGV","count"))
        setcolorder(dat_long,c("bc_old","bc","ensGV","count"))

    }else{
        dat[,V2:=adjust.y(V1),by=1:nrow(dat)]
        chunks=unique(c(seq(from=1,to=nrow(dat),by=1000),nrow(dat)))
        steps=as.data.table(cbind(from=chunks[-length(chunks)],to=chunks[-1]-1))
        steps[nrow(steps),to:=to+1,]
    
        dat_long=data.table()
        for (i in 1:nrow(steps)){
          cat(paste0(i," "))
          from=steps[i]$from
          to=steps[i]$to
          dat_long_tmp=melt(dat[from:to,],id.vars=c("V1","V2"))
          dat_long=rbindlist(list(dat_long,dat_long_tmp[value>0]))
        }
        setnames(dat_long,names(dat_long),c("bc_old","bc","ensGV","count"))
    }
    
    dat_long_filt=dat_long[!grepl("\\+",ensGV)]
    
    dat_long_filt[,ensG:=unlist(strsplit(as.character(ensGV),"\\."))[1],by=1:nrow(dat_long_filt)]
    dat_long_filt[,x:=as.numeric(unlist(strsplit(as.character(bc),"x"))[1]),by=1:nrow(dat_long_filt)]
    dat_long_filt[,y:=as.numeric(unlist(strsplit(as.character(bc),"x"))[2]),by=1:nrow(dat_long_filt)]
    
    dat_long_filt_annot=merge(dat_long_filt,genes,by="ensG",all.x=TRUE)
    dat_long_filt_annot_red=merge(dat_long_filt_annot,sel_bc,by.x=c("x","y"),by.y=c("spot_x","spot_y"))
    
    write.table(dat_long_filt_annot,paste0(dir,"/",sample,"_filtered_red.tsv"),sep="\t",row.names=FALSE,quote=FALSE)
    write.table(dat_long_filt_annot_red,paste0(dir,"/",sample,"_filtered_red_ut.tsv"),sep="\t",row.names=FALSE,quote=FALSE)
    if (plot == TRUE){
    pl=ggplot(dat_long_filt_annot_red[sample(1:nrow(dat_long_filt_annot_red),40000)],aes(y=spot_px_y,x=spot_px_x))+geom_point(size=0.5)+coord_fixed()
    png(paste0(dir,"/",sample,"_filtered_corrected.png"),height=500,width=700)
    print(pl)
    dev.off()
  }
    return(dat_long_filt_annot_red)
}

##### MOB data

In [ ]:
genes=fread("ext_data/ensemble_gene_names_V94.txt")
setnames(genes,names(genes),c("ensGV","ensG","gene"))

In [22]:
dat=prep_hdst_data("MOB","CN13_D2",genes,TRUE)

In [21]:
dat=prep_hdst_data("MOB","CN24_D1",genes,TRUE)

In [ ]:
dat=prep_hdst_data("MOB","CN24_E1",genes,TRUE)

In [ ]:
dat=prep_hdst_data("MOB_nc","CN13_D2_unmodgtf",genes,TRUE)

In [ ]:
dat=prep_hdst_data("MOB_nc","CN24_D1_unmodgtf",genes,TRUE)

In [ ]:
dat=prep_hdst_data("MOB_nc","CN24_E1_unmodgtf",genes,TRUE)

##### Breast cancer data

In [ ]:
genes=fread("ext_data/ensemble_gene_names_human_V96.txt")
setnames(genes,names(genes),c("ensGV","ensG","gene"))

In [ ]:
dat=prep_hdst_data("BC","CN21_BC24350_E2",genes,TRUE)

In [ ]:
dat=prep_hdst_data("BC_nc/","CN21_BC24350_E2_unmodgtf",genes,TRUE)

### Process segmented HDST data

In [ ]:
# for MOB (standard gtf)
dat=fread("MOB_nc/CN13_D2_filtered_red_ut.tsv")
seg=fread("MOB/CellID_Spot_Position_CN13_D2_filtered_red_ut.csv")
tag="CN13_D2"
dir="MOB"

In [ ]:
# for MOB (nc gtf)
dat=fread("MOB_nc/CN13_D2_unmodgtf_filtered_red_ut.tsv")
seg=fread("MOB/CellID_Spot_Position_CN13_D2_filtered_red_ut.csv")
tag="CN13_D2_unmodgtf"
dir="MOB_nc"

In [10]:
# for MOB (nc gtf)
dat=fread("MOB_nc/CN24_D1_unmodgtf_filtered_red_ut.tsv")
seg=fread("MOB_nc/CellID_Spot_Position_CN24_D1_unmodgtf_filtered_red_ut_flipped.csv")
tag="CN24_D1_unmodgtf"
dir="MOB_nc"

In [14]:
# for MOB (nc gtf)
dat=fread("MOB_nc/CN24_E1_unmodgtf_filtered_red_ut.tsv")
seg=fread("MOB_nc/CellID_Spot_Position_CN24_E1_unmodgtf_filtered_red_ut_flipped.csv")
tag="CN24_E1_unmodgtf"
dir="MOB_nc"

In [ ]:
# for BC (nc gtf)
dat=fread("BC_nc/CN21_BC24350_E2_unmodgtf_filtered_red_ut.tsv")
seg=fread("BC/CellID_Spot_Position_CN21_E2_filtered_red_ut_BC_flipped.csv")
tag="CN21_BC24350_E2_unmodgtf"
dir="BC_nc" #previously stored in BC

#### here run

In [15]:
dat_seg_pre=merge(dat,seg[cell_id!=0],by="bc")
dat_seg_pre[,c("x","y","N_bc"):=list(x[1],y[1],length(unique(bc))),by="cell_id"] #assign the x y coordinates of the first barcode to each cell id such that each cell id only has 1 xy coordinate
head(dat_seg_pre)
nrow(dat_seg_pre)

bc,x,y,ensG,bc_old,ensGV.x,count,ensGV.y,gene,spot_px_y,spot_px_x,cell_id,x_centroid,y_centroid,N_bc
1000x187,1000,187,ENSMUSG00000002985,1000x716,ENSMUSG00000002985.16,1,ENSMUSG00000002985.16,Apoe,2295,10077,25146,10067.29,5306.808,10
1000x187,1000,187,ENSMUSG00000020193,1000x716,ENSMUSG00000020193.3,1,ENSMUSG00000020193.3,Zpbp,2295,10077,25146,10067.29,5306.808,10
1000x187,1000,187,ENSMUSG00000020483,1000x716,ENSMUSG00000020483.14,1,ENSMUSG00000020483.14,Dynll2,2295,10077,25146,10067.29,5306.808,10
1000x187,1000,187,ENSMUSG00000025907,1000x716,ENSMUSG00000025907.14,1,ENSMUSG00000025907.14,Rb1cc1,2295,10077,25146,10067.29,5306.808,10
1000x187,1000,187,ENSMUSG00000029635,1000x716,ENSMUSG00000029635.15,1,ENSMUSG00000029635.15,Cdk8,2295,10077,25146,10067.29,5306.808,10
1000x187,1000,187,ENSMUSG00000035202,1000x716,ENSMUSG00000035202.7,1,ENSMUSG00000035202.8,Lars2,2295,10077,25146,10067.29,5306.808,10


[1] 856994

In [16]:
dat_seg=dat_seg_pre[,.(bc=paste0(bc,collapse = " "),count=sum(count),spot_px_y=spot_px_y[1],spot_px_x=spot_px_x[1]),by=c("cell_id","x_centroid","y_centroid","N_bc","gene","x","y")]
head(dat_seg)
length(unique(dat_seg$cell_id))
nrow(dat_seg)
table(dat_seg[,length(unique(cell_id)),by=c("x","y")]$V1) #make sure each coordinate only hase one cell id
table(dat_seg[,length(unique(paste0(x,"_",y))),by=c("cell_id")]$V1) #make sure each cell id only has one coordinate

cell_id,x_centroid,y_centroid,N_bc,gene,x,y,bc,count,spot_px_y,spot_px_x
25146,10067.29,5306.808,10,Apoe,1000,187,1000x187,1,2295,10077
25146,10067.29,5306.808,10,Zpbp,1000,187,1000x187,1,2295,10077
25146,10067.29,5306.808,10,Dynll2,1000,187,1000x187,1,2295,10077
25146,10067.29,5306.808,10,Rb1cc1,1000,187,1000x187,1,2295,10077
25146,10067.29,5306.808,10,Cdk8,1000,187,1000x187 996x186 999x189,3,2295,10077
25146,10067.29,5306.808,10,Lars2,1000,187,1000x187,1,2295,10077


[1] 22229

[1] 684443


    1 
22229 


    1 
22229 

In [17]:
write.table(dat_seg,paste0(dir,"/",tag,"_filtered_red_ut_segmented.tsv"),sep="\t",quote=FALSE,row.names = FALSE)

### Process binned HDST data

In [2]:
#for MOB (standard gtf)
bin_sizes=c("5x","10x","20x","38x","38x-thin")
dir="MOB_binned"

In [ ]:
#for MOB (nc gtf)
bin_sizes=c("5x")
dir="MOB_binned_nc"

In [3]:
#for MOB (nc gtf) new sample
bin_sizes=c("5x")
dir="MOB_binned_nc/E1"

In [7]:
#for MOB (nc gtf) new sample
bin_sizes=c("5x")
dir="MOB_binned_nc/D1"

In [2]:
#for BC (standard gtf)
bin_sizes=c("5x")
dir="BC_binned"

In [2]:
#for BC (nc gtf)
bin_sizes=c("5x")
dir="BC_binned_nc"

In [3]:
#for BC (nc gtf)
bin_sizes=c("5x")
dir="BC_binned_nc/C1"

In [7]:
#for BC (nc gtf)
bin_sizes=c("5x")
dir="BC_binned_nc/D1"

In [8]:
dat_all_bin=data.table()
for (bin in bin_sizes){
    dat_file=list.files(path = dir,pattern = paste0("hdst.*lowres-",bin,".csv.*"),full.names = TRUE)
    bins_file=list.files(path = dir,pattern = paste0("hdst.*lowres-",bin,"-bins.csv.*"),full.names = TRUE)
    print(dat_file)
    dat=fread(dat_file)
    bins=fread(bins_file)
    dat[,x:=bins$x,]
    dat[,y:=bins$y,]
    dat_long=melt(dat,id.vars = c("x","y"),variable.name = "gene",value.name = "count")
    dat_long=dat_long[count>0]
    dat_long[,bin:=bin,]
    dat_long[,bc:=paste0(x,"_",y),]
    dat_all_bin=rbindlist(list(dat_all_bin,dat_long))
}

[1] "BC_binned_nc/D1/hdst-breast-cancer-D1-lowres-5x.csv.gz"


In [9]:
head(dat_all_bin)

x,y,gene,count,bin,bc
241,67,TSPAN6,1,5x,241_67
76,134,DPM1,1,5x,76_134
94,80,DPM1,1,5x,94_80
142,155,DPM1,1,5x,142_155
144,156,DPM1,1,5x,144_156
185,16,DPM1,1,5x,185_16


In [10]:
write.table(dat_all_bin,paste0(dir,"/hdst-lowres.tsv"),sep="\t",quote=FALSE,row.names=FALSE)

### Process standard ST

In [90]:
prep_st=function(mat){
    mat[,x:=unlist(strsplit(V1,"x"))[1],by=1:nrow(mat)]
    mat[,y:=unlist(strsplit(V1,"x"))[2],by=1:nrow(mat)]
    mat_long=melt(mat[,-c("V1"),],id.vars = c("x","y"),variable.name = "gene",value.name = "count")
    return(mat_long[count>0])
}

In [91]:
st_dat=fread("MOB_STST/Rep4_MOB1x.csv")

Warning message in fread("MOB_STST/Rep4_MOB1x.csv"):
"Detected 15941 column names but the data has 15942 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file."

In [92]:
st_long=prep_st(st_dat)

In [94]:
tail(st_long)

x,y,gene,count
16.105,29.003,Mx1,1
16.105,29.003,Cenpa,1
16.105,29.003,Snora17,1
16.984,29.022,Nlrp5,1
16.984,29.022,Sned1,1
16.984,29.022,Gm933,1


In [95]:
write.table(st_long,"MOB_STST/Rep4_MOB1x_long.tsv",sep="\t",quote=FALSE,row.names=FALSE)

### Annotate barcodes with anatomic features

In [ ]:
library(data.table)
library(raster)
library(rgeos)
library(ggplot2)

In [ ]:
setwd(data_dir) #set wd to directory containing the polygon file as well as the barcodes under tissue  file

In [ ]:
annotate_bc=function(sample,plot=TRUE,max_dist=Inf,n_chunks=NULL,flip=TRUE){
  #read data
  poly=fread(paste0(sample,"_annotations.txt"))
  print(nrow(poly))
  bc=fread(paste0(sample,"_barcodes_under_tissue.tsv"))
  poly[value=="",value:="Unknown"]
  if (any("polygon"%in%names(poly))){setnames(poly,"polygon","x_y")}

  
  #convert polygons table into list
  poly_long=poly[,.(x_y=strsplit(x_y," "),value),by=1:nrow(poly)]
  poly_list=apply(poly_long,1,function(x){r=lapply(strsplit(unlist(x["x_y"]),","),as.numeric);r=lapply(r,function(x){if(length(x)==1){x=c(x,NA)};return(x)});r=t(as.data.table(r));colnames(r)=c("x","y");return(na.omit(r))})
  names(poly_list)=poly_long$value
  
  #make list of polygon objects
  sp=rapply(poly_list, Polygon,hole=FALSE, how = "replace")
  sp=lapply(1:length(sp), function(i) {Polygons(sp[i], as.character(i))})
  
  #make spatial poligons object
  pols=SpatialPolygons(sp)
  plot(pols)
  
  #prepare HD_ST coordinates (need to be mirrored at y-axis in some samples)
  if (flip==TRUE){
    print("Flipping y axis.")
    flipped_bc=bc[,.(spot_px_x=spot_px_x,spot_px_y=-(spot_px_y-min(spot_px_y))+(max(spot_px_y))),]
  }else{
    print("Not flipping y axis.")
    flipped_bc=bc[,.(spot_px_x=spot_px_x,spot_px_y=spot_px_y),]
  }
  
  #overlap polygons and HD_ST coordinates
  nr=nrow(flipped_bc)
  print(nr)

    if (!is.null(n_chunks)){
      chunks_size=floor(nr/n_chunks)
      e=data.table()
      for (i in 0:n_chunks){
        start=i*chunks_size+1
        end=ifelse(i==n_chunks,nr,(i+1)*chunks_size)
        if (start > nr){break}
        print(paste0("Processing chunk ",start," to ",end))
        e1 = as.data.table(extract(pols, flipped_bc[start:end]))
        e1[,point.ID:=point.ID+(start-1),]
        e=rbindlist(list(e,e1))
      }
    }else{
      e = as.data.table(extract(pols, flipped_bc))
    }  
  
  e[,poly:=poly.ID,]
  e[,poly.ID:=names(poly_list)[poly],]
  
  #find nearest polygons for HD_ST coordinates that don't fall into a polygon
  print("Now assigning missing.")
  missing_id=e[is.na(poly.ID)]$point.ID
  missing=flipped_bc[missing_id,]
  missing[,point.ID:=missing_id,]
  sp_pts=SpatialPoints(missing[,c("spot_px_x", "spot_px_y"),])
  dist=gDistance(sp_pts,pols,byid = TRUE)
  if(max_dist!=Inf){
  dist=apply(dist,c(1,2),function(x){ifelse(x>max_dist,NA,x)})
  missing[,np:=apply(dist,2,function(x){res=which.min(x);ifelse(length(res)>0,res,NA)})]
  missing=missing[!is.na(np)]
  }else{
    missing[,np:=apply(dist,2,which.min)]
  }
  missing[,poly:=np,]
  missing[,poly.ID:=names(poly_list)[np],]
  
  #combine annotations from primary and secondary assignment
  e_complete=rbindlist(list(e[!is.na(poly.ID)],missing[,c("point.ID","poly.ID","poly"),]),use.names = TRUE)
  
  #merge with original HD_ST coordinates file
  bc[,point.ID:=1:nrow(bc),]  
  bc_annot=unique(merge(bc,e_complete,by="point.ID",all.x=TRUE))  
  bc_annot[is.na(poly.ID),poly.ID:="missing",]
  bc_annot=bc_annot[!duplicated(point.ID)] #just take the first entry if a barcode is assigned to several annotations
  
  if (plot == TRUE){
    pl=ggplot(bc_annot,aes(y=spot_px_y,x=spot_px_x,col=poly.ID))+geom_point(size=0.5)+coord_fixed()
    png(paste0(sample,"_bc_annot.png"),height=500,width=700)
    print(pl)
    dev.off()
  }
  write.table(bc_annot,paste0(sample,"_barcodes_under_tissue_annot.tsv"),sep="\t",quote=FALSE,row.names=FALSE)
  return(bc_annot)
}

##### MOB

In [ ]:
bc=annotate_bc("CN13_D2",TRUE)

#### Breast cancer

In [ ]:
bc=annotate_bc("CN21_BC24350_E2",TRUE,max_dist = 5,n_chunks = 10,flip = FALSE)